In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import timedelta, datetime
from scipy import stats
from scipy.stats import shapiro
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_regression

warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('test.csv')
df.head()

## Data Exploration

In [ ]:
train_df = pd.read_csv('train.csv')
train_df.head()

In [ ]:
train_df.columns

In [ ]:
train_df.info()

In [ ]:
train_df.shape

In [ ]:
##Find missing values for each column
train_df.isnull().sum().groupby(train_df.columns).agg(['sum']).sort_values(by='sum', ascending=False)
## Find the missing percent
train_df.isnull().mean().groupby(train_df.columns).agg(['mean']).sort_values(by='mean', ascending=False)



## Response Variable Analysis
- forward_returns
- market_forward_excess_returns
- risk_free_rate

In [ ]:
response_col = ['forward_returns', 'market_forward_excess_returns', 'risk_free_rate']
response_col = [col for col in train_df.columns if col in response_col]

#Plot the distributions
fig, axs = plt.subplots(4, len(response_col),figsize=(30,10))
for i, col in enumerate(response_col):
  data = train_df[col].dropna()

  print(f'{col} Stats')
  print(f'Skewness: {data.skew():3f}')
  print(f'Kurtosis: {data.kurtosis():3f}')

  sns.histplot(data=data, ax=axs[0,i])
  axs[0,i].axvline(data.mean(), linestyle='--', color='red', label=f'{col} Mean:{data.mean():.4f}')
  axs[0,i].axvline(data.median(), linestyle='--', color='blue', label=f'{col} Median:{data.median():.4f}')
  axs[0,i].set_title(f'{col} Distribution')
  axs[0,i].legend()

  sns.histplot(data=data,stat='density',ax=axs[1,i])
  sns.kdeplot(data=data, color='red', lw=2, ax=axs[1, i])
  axs[1,i].set_title(f'{col} Histogram with Density Plot')

  sns.lineplot(data=data, ax=axs[2,i])
  axs[2,i].set_title(f'{col} Time Series Plot')

  #Q-Q Plot
  stats.probplot(data, dist='norm', plot=axs[3,i])



  ## Shapire-Wilkins test
  statistic, p = shapiro(data)
  print(f"Shapiro-Wilk Test: Statistic={round(statistic,3)}, p={round(statistic,3)}, is_normal={p>0.05}\n")



plt.tight_layout()
plt.savefig('response_analysis.png')
plt.show()



## Analysis
Based on numeric and graphic representations of the distribution of hte response variable, we can identify that they're not normally distributed.


**Note:** Can't use model that rely on normality assumption.

## Feature Analysis


In [ ]:
feature_map = {
    "Dummy/Binary features(D)": [col for col in train_df.columns if col.startswith('D') and col[1:].isnumeric()],
    "Macro Economic features(E)": [col for col in train_df.columns if col.startswith('E') and col[1:].isnumeric()],
    "Interest Rate features(I)": [col for col in train_df.columns if col.startswith('I') and col[1:].isnumeric()],
    "Market Dynamics/Technical features(M)": [col for col in train_df.columns if col.startswith('M') and col[1:].isnumeric()],
    "Price/Valuation features(P)": [col for col in train_df.columns if col.startswith('P') and col[1:].isnumeric()],
    "Sentiment features(S)": [col for col in train_df.columns if col.startswith('S') and col[1:].isnumeric()],
    "Volatility features(V)": [col for col in train_df.columns if col.startswith('V') and col[1:].isnumeric()]
}
for group_name, feature_group in feature_map.items():
    if not feature_group:
        continue

    num_cols = 4
    num_rows = int(np.ceil(len(feature_group) / num_cols))

    fig, axs = plt.subplots(num_rows, num_cols, figsize=(20, 4 * num_rows))
    axs = axs.ravel() if len(feature_group) > 1 else [axs]
    fig.suptitle(f"{group_name}", fontsize=16, y=1.02)

    for idx, col in enumerate(feature_group):
        data = train_df[col].dropna()

        axs[idx].hist(x=data, bins=30, color='skyblue', edgecolor='black')
        axs[idx].axvline(data.mean(), linestyle='--', color='red', label=f'Mean: {data.mean():.3f}')
        axs[idx].axvline(data.median(), linestyle='--', color='blue', label='Median')
        axs[idx].set_xlabel(col)
        axs[idx].set_ylabel('Frequency')
        axs[idx].set_title(f'{col} Distribution')
        axs[idx].legend()


    # Remove unused subplots
    for j in range(len(feature_group), len(axs)):
        fig.delaxes(axs[j])


    plt.tight_layout()
    plt.show()

### Observations

Dummy Variables (D): are binary observations. Furthermore there is an imbalance of these observations, having more False's as Outcomes than True.

* * *
Economics (E):

- Skewness
  1. E1,E4, E6, E9, E11-E14, E19: Right Skewed, meaning most values are clustered around 0. Presence of extreme outliers.
  2. E8,E16, E17: showcase light left skewness or bimodality
- Sparse Features
  1. E4,E6, E11-E14, have small ranges with majority of value clustering around 0.
  - Could represent low variance.
  - Might need scaling adjustments or removal if they are not significant
- Uniform
  1. E5, E10, & E15 looks to be a uniform distribution
  - Might be useful in capturing non-linear distributions.
- Mean vs. Median
  1. Distributions showcase large gap between mean and median, reinforces high-skewness
  - Mean > Median: positive skew
  - Might require log-transformation or standardizing.
- Potential Outliers
  - Several variables showcases extreme spikes, or skewness, indicating extreme outliers
  - Might need to use impute them using winzorized means.
- Normal Distributions
  - E3, E16, E17, E18, and E20 approximately normally distributed.
- Modeling Implications:
  - High-Skewness: Log-Scaling or some other transformations
  - Biomodal: Clustering algos
  - Normal: Linear Models
  - Low-Variance: may be dropped or might need to combine them.

* * *
Interest(I)
- Skewness:
  1. I2: Right-skewed, potentially indicating the presence of outliers on the higher end.
  2. I7, I2: Left-skewed, suggesting potential outliers on the lower end or a concentration of values towards the higher end.
- Uniform Distribution:
  - I3, I4, I6, I7, I8: These features appear to have a relatively uniform distribution, where values are spread somewhat evenly across the range.
- Normal Distribution:
  - I5, I9: These features seem to follow an approximately normal distribution.
- Skewness and Mean vs. Median:
  - The presence of skewness suggests the possibility of high or medium outliers, which could impact models sensitive to extreme values.
  - The gap between the mean and median in some distributions reinforces the observation of skewness and potentially low variance in certain features.
- Modeling Implications:
  - Skewed distributions might require transformations (e.g., log transformation) to make them more symmetrical, which can improve the performance of some models.
  - Features with low variance or uniform distributions might require careful consideration. They might not be highly informative on their own but could be useful in combination with other features or in certain model types.

* * *
Market(M)
- Skewness:
  1. M1, M10, M12, M13, M14, M15, M16, M17, M18, M2, M3, M4, M5, M6, M7, M8, M9: Many of the Market features exhibit varying degrees of skewness, both left and right. This indicates that the distribution of values is not symmetrical for these features.
- Multimodality:
  - Some features like M2, M3, and M6 show signs of multimodality (having multiple peaks in their distribution), suggesting that there might be distinct subgroups or regimes within the data that these features represent.
- Potential Outliers:
  - The presence of skewness and long tails in some distributions suggests the potential for outliers in the Market features. These outliers could disproportionately influence certain models.
- Mean vs. Median:
  - The difference between the mean and median in skewed distributions highlights the impact of outliers on the mean.
- Modeling Implications:
  - Skewed and multimodal distributions might require non-linear models or models that are robust to non-normal data.
  - Multimodal features could indicate the need for clustering or segmentation of the data based on these features.
  - Outliers might need to be handled through robust scaling methods, winsorization, or removal, depending on their nature and impact on the model.


### Calculating Outliter Proportions for response and features

#### Response

In [ ]:
def outlier_prop(df, response_col):
  outlier_props = {}

  for response in response_col:
    data = df[response].dropna()

    ### Calculate Metrics
    q1 = data.quantile(0.25)
    q3 = data.quantile(0.75)

    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    outliers = data[(data < lower_bound) | (data > upper_bound)]
    outlier_prop = len(outliers) / len(data)
    outlier_props[response] = outlier_prop

  return outlier_props

### Outlier props
outlier_prop = outlier_prop(train_df, response_col)
outlier_response_df = pd.DataFrame(list(outlier_prop.items()), columns=['Response', 'Outlier Proportion'])
outlier_response_df


In [ ]:
### Plot the props
plt.figure(figsize=(8,4))
sns.barplot(x='Response', y='Outlier Proportion', data=outlier_response_df, palette='viridis')
plt.title('Outlier Proportion by Response Variable')
plt.show()

In [ ]:
## Stationary
from statsmodels.tsa.stattools import adfuller

for col in response_col:
  data = train_df[col].dropna()
  result = adfuller(data)
  print(f'{col} Stats')
  print(f'ADF Statistic: {result[0]:.4f}')
  print(f'p-value: {result[1]:.4f}, is_stationary: {result[1] < .05}\n')




#### Features

In [ ]:
def calc_outlier_proportion(series):
    """Calculate outlier proportion for a numeric column using IQR."""
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outliers = ((series < lower) | (series > upper)).sum()
    return outliers / series.notna().sum() if series.notna().sum() > 0 else np.nan

outlier_summary = []

for group_name, feature_group in feature_map.items():
    for col in feature_group:
        outlier_prop = calc_outlier_proportion(train_df[col])
        outlier_summary.append({
            'Group': group_name,
            'Feature': col,
            'Outlier Proportion': outlier_prop
        })
outlier_summary_df = pd.DataFrame(outlier_summary)

## Group level summary
group_avg = (
    outlier_summary_df.groupby("Group")["Outlier Proportion"]
    .mean()
    .sort_values(ascending=False)
)
group_avg

In [ ]:
import re
### Plot the proportion by group_avg
pattern = r'\((.*?)\)'
short_initials = [re.search(pattern, name).group(1) for name in group_avg.index]

plt.figure(figsize=(10,4))
plt.figure(figsize=(10, 4))
sns.barplot(x=short_initials, y=group_avg.values, palette="viridis")

plt.title('Average Outlier Proportion by Feature Group')
plt.xlabel('Feature Group (Short Code)')
plt.ylabel('Average Outlier Proportion')

plt.tight_layout()
plt.show()

#### Observation

Forward_returns & Market_Forward_returns $\approx 6.6. \%$



# Statistical Testing for Assumptions

### Stationary Test(ADF) Response
$H_0$: non-stationary \
$H_A$: stationary

If p< $\alpha = 0.05$ (Stationary)

In [ ]:
for col in response_col:
  data = train_df[col].dropna()
  result = adfuller(data)

  if result[1] >= .05:
    print(f'{col} is non-stationary, p-value:{result[1]}')

### Observations:
- forward_returns, market_forward_returns: Stationary
- risk_free_rate : Non-Stationary

### Stationarity (Features)

In [ ]:
non_stationary = {}


for feature in train_df.columns[1:-1]:
  col = train_df[feature].dropna()
  result = adfuller(col)

  if result[1] >= .05:
    non_stationary[feature] = (result[0], result[1])
    print(f'{feature} is non-stationary, p-value:{result[1]}')

In [ ]:
## Count of non-stationary features
print(f'Number of non-stationary features: {len(non_stationary)}')


## Calculate the proportion of non-stationary features
for prefix in ['D','E','I','M','P','S']:
  all_cols = [col for col in train_df.columns if col.startswith(prefix) and col[1:].isnumeric()]
  non_stat = [col for col in all_cols if col in non_stationary]

  print(f"{prefix}, Count:{len(non_stat)}, Prop: {100 * len(non_stat) / len(all_cols) if len(all_cols) > 0 else 0:.3f}%")

## Conclusion:
Non-stationary features: ['E10', 'E11', 'E12', 'E2', 'E20', 'E3', 'E5', 'E6', 'E9', 'I4', 'I5', 'I8', 'I9', 'M13', 'M14', 'M16', 'M17', 'M18', 'P10', 'P11', 'P9', 'V11', 'V8', 'risk_free_rate']

Need to adjusted or removed


Stationary: Property or an assumption, that the statistical properties do not change over time.

## AutoCorrelation

#### Response


In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

for col in ['market_forward_excess_returns']:
  time_series_data = train_df[col].dropna()

  # Plot ACF
  fig, ax = plt.subplots(figsize=(10, 5))
  plot_acf(time_series_data, ax=ax, lags=40)
  ax.set_title(f'Autocorrelation Function (ACF) for {col}')
  ax.set_xlabel('Lags')
  ax.set_ylabel('Autocorrelation')
  plt.savefig('autocorrelation.png')
  plt.show()

  # Plot PACF
  fig, ax = plt.subplots(figsize=(10,5))
  plot_pacf(time_series_data, ax=ax, lags=40)
  ax.set_title(f'Partial Autocorrelation Function (PACF) for {col}')
  ax.set_xlabel('Lags')
  ax.set_ylabel('Partial Autocorrelation')

  plt.savefig('partial_autocorrelation.png')
  plt.show()

### Observations
PACF and ACF plots highlights how much a variable is related to its past values.

- Forward_returns and market_forward excess_return, the plot suggests that current observation value is mainly related to values for the recent past, this relationship also fades very quickly thus highlighting that these variables are relatively stable over time
- For risk_free rate: the plots suggest that past values have a longer-lasting impact, and the relationship decays more slowly; thus confirming that this reponse_variable is non-stationary.

### Normality (Features)

$H_0$: Not normal \
$H_A$: Normal

If p-val $ < \alpha = .05$ (Normal)






In [ ]:
non_normal = {}
for feature in train_df.columns[1:len(train_df) - 1]:
  result = shapiro(train_df[feature].dropna())
  if result[1] >= .05:
    non_normal[feature] = (result[0], result[1])
    print(f'{feature} is not normal, p-value:{result[1]:.3f}')
  print(f"{feature} is normal, p-value:{result[1]}")

### Conclusion
Based on the p-value for all the features we can assume that all the distributions are normal, even the graph suggest otherwise on some variables.

### Homoscedasiticity (Constant Variance) (Response)

$H_0$: Heteroscedasiticity \
$H_A$: Homoscedasiticity

If p-val $ < \alpha = .05$ (Homoscedasiticity)

In [ ]:
from statsmodels.stats.diagnostic import het_arch

for col in response_col:
  data = train_df[col].dropna()
  result = het_arch(data)
  print(f'{col} Stats')
  print(f'p-value: {result[1]:.4f}, is_homoscedastic: {result[1] < .05}\n')

In [ ]:
non_homoscedastic = {}
for feature in train_df.columns[1:len(train_df) - 1]:
  result = het_arch(train_df[feature].dropna())
  if result[1] >= .05:
    non_homoscedastic[feature] = (result[0], result[1])
    print(f'{feature} is not homoscedastic, p-value:{result[1]:.3f}')

### Conclusion:

From the previous conclusion, we concluded that most all the features were normal, thus, we inherently assume that they are homoscedastic.

### Multicolinearity
VIF < 8 (ideally < 3) means acceptable

If high apply PCA or drop correlated features

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
features = [col for col in train_df.columns if col not in ["date_id", "forward_returns", "risk_free_rate", "market_forward_excess_returns"]]
features_df = train_df[features].dropna()

vif_data = pd.DataFrame()
vif_data["feature"] = features_df.columns
vif_data["VIF"] =[variance_inflation_factor(features_df.values, i)
                    for i in
range(len(features_df.columns))]

vif_data.sort_values(by="VIF", ascending=False)

### Observation
Based on the dataframe above we can either drop the high VIF columns greater than 8 or we can utilize PCA to reduce correlated features.

### How to Interpret the Table
The higher the VIF value, greater the feature correlation with another feature.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

corr_matrix = features_df.corr()
strong_corr_matrix = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
      if abs(corr_matrix.iloc[i, j]) > 0.8:
        strong_corr_matrix.append(
            (corr_matrix.columns[i], corr_matrix.columns[j], abs(corr_matrix.iloc[i,j]))
        )
strong_corr_df = pd.DataFrame(strong_corr_matrix, columns=['Feature 1', 'Feature 2', 'Correlation'])
print(strong_corr_df)



In [ ]:
max_weak = abs(corr_matrix) < 0.5

plt.figure(figsize=(30,15))
sns.heatmap(
    corr_matrix,
    annot=False,
    cmap='coolwarm',
    mask=max_weak,
    linewidths=0.5
)
plt.title('Strong Correlation Heatmap')
plt.show()

## Cross Correlation


In [ ]:
from statsmodels.tsa.stattools import ccf

feature_cols = [col for col in train_df.columns if col not in response_col]
ccf_results = {}
for feature in feature_cols:
  for response in response_col:
    temp_df = train_df[[feature, response]].dropna()
    if len(temp_df) > 0:
      cross_corr = ccf(temp_df[feature], temp_df[response], adjusted = False)

      ccf_results[(feature, response)] = cross_corr
      #Plotting
      plt.figure(figsize=(10,5))
      plt.stem(cross_corr)
      plt.title(f'Cross-Correlation between {feature} and {response}')
      plt.xlabel('Lag')
      plt.ylabel('Cross-Correlation')
      plt.grid(True)
      plt.show()

In [ ]:
ccf_summary = []
for (feature, response), corr_values in ccf_results.items():
    for lag, value in enumerate(corr_values):
        ccf_summary.append({
            'Feature': feature,
            'Response': response,
            'Lag': lag,
            'CrossCorrelation': value
        })
ccf_df = pd.DataFrame(ccf_summary)
ccf_df = ccf_df[ccf_df['Feature'] != 'date_id'] # Remove date_id

ccf_df = ccf_df.loc[ccf_df['CrossCorrelation'].abs().sort_values(ascending=False).index]
ccf_df.head(10) # Top 10 Cross correlated feat

### Do T-test
-

In [ ]:
ccf_df_group = ccf_df.groupby(['Feature','Response']).agg({
    'Lag': 'mean',
    'CrossCorrelation': 'mean'
}).reset_index()

ccf_df_group

### Observation of Cross-Correlation

- The dataframe above showcases the mean lag and mean cross-correlation for each feature-response pair.

- The mean cross-correlation values accross all lags for most feature-response pairs are very close to 0.

- This suggests that on average accross all lags, there isn't a strong linear relationship between the features and the response variables.



### Insights
**Response Variables**:
 - forward_returns and market_forward_excess_returns are stationary but not normally distributed and contain outliers; risk_free_rate is non-stationary.

**Feature Properties**:

- Many features are non-stationary (especially macroeconomic and interest rate groups) and show strong multicollinearity, particularly within binary, rate, and volatility groups.

**Cross-Correlation**:
 - Most feature, response pairs have near zero mean correlation, but some show strong correlations at specific lags — features with negative lag correlation may act as leading indicators.

**Missing Values**:
 - Present in several features need imputation or removal.

### Modeling Recommendations

**Time Series Baselines**:

 - ARIMA for stationary or differenced data.

 - Exponential Smoothing (Holt-Winters) for trend/seasonal components.

**Feature-Based Models**:

- Ridge / Lasso Regression to handle multicollinearity.

 - Tree-based models (Random Forest, XGBoost, LightGBM) for robustness to non-normality and nonlinear relationships.
